# Validation Set 1: diffuPy + PathMe  

In [1]:
import os
dir_path = os.path.dirname(os.path.realpath('__file__'))

In [2]:
from openpyxl import load_workbook
from collections import defaultdict
import pybel
import pybel_tools as pbt
import networkx as nx

from pybel.dsl import Abundance, BiologicalProcess, CentralDogma, ListAbundance, Reaction

from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL, PATHME_DIR

### Load Data Set 1: Input Scores

In [3]:
def munge_labels(label):
    """Process ene"""
    remove_set = ['*', ' ', '|', '-', '"', "'"]
    
    label = str(label).lower()
    
    for symb in remove_set:
        if symb in label:
            label = label.replace(symb, '')
    
    if '/' in label:
        label = tuple(set(label.split('/')))
        if len(label) == 1:
            label = label[0]
    
    return label


def parse_set1(path):
    
    wb = load_workbook(filename = path)

    sheet_titles = []
    omics_data = defaultdict(lambda:defaultdict(lambda:set()))
    omics_labels = defaultdict(lambda:set())

    for sheet in wb:
        cell_value = sheet['A3'].value

    #     if "Expression data (FC) of the differentially expressed" in sheet['A1'].value:
    #         sheet_title = sheet['A1'].value.split("Expression data (FC) of the differentially expressed ",1)[1]
    #         sheet_title = sheet_title.split(" of HepG2 cells after treatment with ")
    #         sheet_title[1] = sheet_title[1].replace(". Statistical significance (p value < 0.05) is indicated.", "").replace(" CsA for", "")
    #         sheet_titles.append(sheet_title)

        if cell_value and ("Significant " in cell_value or "Metabolite" == cell_value):
            if  cell_value == "Metabolite":
                sheet_title = ("Metabolite", '3 µM', ' 24h or 72h')
                min_row = 3

            else:
                sheet_title = cell_value.split("Significant ",1)[1]
                sheet_title = sheet_title.split(" CsA ")
                sheet_title.append(sheet_title[1].split(" ")[0] + ' h')
                sheet_title[1] = sheet_title[0].split(" ")[1]+ ' µM'
                sheet_title[0] = sheet_title[0].split(" ")[0]
                min_row = 4

            for col in sheet.iter_cols(min_row=min_row):
                col_label = col[0].value
                sheet_omic = sheet_title[0]

                if col_label in ['MicroRNA', 'hgnc symbol', 'Metabolite']:
                    omics_labels[sheet_omic.lower()].update(munge_labels(cell.value) for cell in col[1:])

            sheet_titles.append(sheet_title)

    return omics_labels
    
dataset1_omics_labels = parse_set1(os.path.join(dir_path, 'validation', 'set1.xlsx'))
dataset1_omics_labels

defaultdict(<function __main__.parse_set1.<locals>.<lambda>()>,
            {'genes': {'',
              'arhgap21',
              'txndc17',
              'sod1',
              'hbg2',
              'fgfbp3',
              'cers4',
              'ifi6',
              'prmt6',
              'znf195',
              'eef1d',
              'rtcd1',
              'ppt1',
              'prr14',
              'hif1a',
              'plrg1',
              'trim25',
              'tmem14a',
              'hebp2',
              'cdipt',
              'mdm4',
              'mtfr1',
              'ube2w',
              'dus1l',
              'klhdc2',
              'bcl9l',
              'ripk2',
              'klhl21',
              'clpx',
              'ttc13',
              'glud1',
              'c11orf67',
              'abcg8',
              'hlf',
              'cyp20a1',
              'decr1',
              'bbs2',
              'arl8a',
              'cmtm6',
              'rusc2',
    

In [4]:
print(f'Total number of genes: ({len(dataset1_omics_labels["genes"])})')

print(f'Total number of metabolites: ({len(dataset1_omics_labels["metabolite"])})')

print(f'Total number of miRNAs: ({len(dataset1_omics_labels["micrornas"])})')

total_entities = len(dataset1_omics_labels["metabolite"]) + len(dataset1_omics_labels["micrornas"]) + len(dataset1_omics_labels["genes"])

print(total_entities)

Total number of genes: (4942)
Total number of metabolites: (21)
Total number of miRNAs: (100)
5063


## PathMe Imports

### Graph Universe

In [5]:
# Modify Python Typing arguments if you are using Python < 3.6
# PyBEL union method cannot be used because empty nodes are discarded
def get_nodes_in_database(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return {
        node
        for network in database_networks
        for node in network.nodes()
    }

kegg_nodes = get_nodes_in_database(KEGG_BEL)
reactome_nodes = get_nodes_in_database(REACTOME_BEL)
wikipathways_nodes = get_nodes_in_database(WIKIPATHWAYS_BEL)

#### Manual Curation
Since some of the entities were not categorized with their corresponding BEL function (e.g., abundance, gene) due to abnormalities in harmonization, we have manually curated them and assigned them to the right modality.

This data can be used by the original database to correct the node label in their corresponding pathway files.

In [6]:
# Entities in WikiPathways that required manual curation
WIKIPATHWAYS_BIOL_PROCESS = {'lipid biosynthesis', 'hsc survival', 'glycolysis & gluconeogenesis', 'triacylglyceride  synthesis', 'wnt canonical signaling', 'regulation of actin skeleton', 'fatty acid metabolism', 'mrna processing major splicing pathway', 'senescence', 'monocyte differentiation', 'pentose phosphate pathway', 'ethanolamine  phosphate', 'hsc differentiation', 'actin, stress fibers and adhesion', 'regulation of actin cytoskeleton', 's-phase progression', 'g1-s transition', 'toll-like receptor signaling pathway', 'regulation of  actin cytoskeleton', 'proteasome degradation', 'apoptosis', 'bmp pathway', 'ampk activation', 'g1/s checkpoint arrest', 'mapk signaling pathway', 'chromatin remodeling and  epigenetic modifications', 'wnt signaling pathway', 'ros production', 'erbb signaling pathway', 'shh pathway', 'inflammation', 'dna replication', 'mrna translation', 'oxidative stress', 'cell cycle checkpoint activation', 'gi/go pathway', 'wnt pathway', 'g1/s transition of mitotic cell cycle', 'modulation of estrogen receptor signalling', 'dna repair', 'bmp canonical signaling', 'igf and insuline signaling', 'unfolded protein response', 'cell death', 'p38/mapk  pathway', 'glycogen metabolism', 'gnrh signal pathway', 'the intra-s-phase checkpoint mediated arrest of cell cycle progression', 'tca cycle', 'mtor protein kinase signaling pathway', 'proteasome  degradation pathway', 'morphine metabolism', 'hsc aging', 'gastric pepsin release', 'parietal cell production', 'prostaglandin pathway', 'cell cycle (g1/s)  progression', 'notch pathway', 'g2/m progression', 'wnt signaling', 'cell adhesion', 'cell cycle progression', 'egfr pathway', 'cell cycle', 'angiogenesis', 'g2/m-phase checkpoint', 'hsc self renewal', '26s proteasome  degradation', 'mapk signaling', 'immune system up or down regulation', 'm-phase progression', 'insulin signaling', 'nf kappa b pathway', 'cell cycle  progression', 'gi pathway', 'cd45+ hematopoietic-    derived cell    proliferation', "kreb's cycle", 'glycogen synthesis', 'apoptosis pathway', 'g1/s progression', 'inflammasome activation', 'melanin biosynthesis', 'proteasomal degradation', 'g2/m checkpoint arrest', 'g1/s cell cycle transition', 'dna damage response', 'gastric histamine release'}
WIKIPATHWAYS_METAB = {'2,8-dihydroxyadenine', '8,11-dihydroxy-delta-9-thc', 'adp-ribosyl', 'cocaethylene', 'dhcer1p', 'ecgonidine', 'f2-isoprostane', 'fumonisins b1', 'iodine', 'l-glutamate', 'lactosylceramide', 'methylecgonidine', 'n-acetyl-l-aspartate', 'nad+', 'nadph oxidase', 'neuromelanin', 'nicotinic acid (na)', 'nmn', 'pip2', 'sphingomyelin', 'thf'}
WIKIPATHWAYS_NAME_NORMALIZATION = {"Ca 2+": "ca 2+", "acetyl coa": "acetyl-coa", "acetyl-coa(mit)": "acetyl-coa", "h20": "h2o"}

# Entities in Reactome that required manual curation
BLACK_LIST_REACTOME = {"5'"}
REACTOME_PROT = {'phospho-g2/m transition proteins', 'integrin alpha5beta1, integrin alphavbeta3, cd47', 'food proteins', 'activated fgfr2', 'adherens junction-associated proteins', 'pi3k mutants,activator:pi3k', 'prolyl 3-hydroxylases', 'gpi-anchored proteins', 'c3d, c3dg, ic3b', 'c4s/c6s chains', 'activated fgfr1 mutants and fusions', 'activated fgfr3 mutants', 'protein', 'cyclin a2:cdk2 phosphorylated g2/m transition protein', 'c4c, c3f', 'activated raf/ksr1', 'activated fgfr1 mutants', 'g2/m transition proteins', 'lman family receptors', 'cyclin', 'usp12:wdr48:wdr20,usp26', 'proteins with cleaved gpi-anchors', 'activated fgfr2 mutants', 'c4d, ic3b', 'c5b:c6:c7, c8, c9', 'cyclin a1:cdk2 phosphorylated g2/m transition protein', 'genetically or chemically inactive braf', 'il13-downregulated proteins', 'activated fgfr4 mutants', 'rna-binding protein in rnp (ribonucleoprotein) complexes', 'effector proteins', 'usp3, saga complex', 'dephosphorylated "receiver" raf/ksr1'}

#### Methods used in this notebook to process node information and create a set for each modality/database

In [7]:
def process_reactome_multiple_genes(genes):
    """Process a wrong ID with multiple identifiers"""
    gene_list = []
    for counter, gene in enumerate(genes):
        
        # Strip the ' gene' prefix
        gene = gene.strip().strip(' gene').strip(' genes')
        
        # First element is always OK
        if counter == 0:
            gene_list.append(gene)
        
        # If the identifier starts the same than the first one, it is right
        elif gene[:2] == genes[0][:2]:
            gene_list.append(gene)
        
        # If the identifier is longer than 2 it is a valid HGNC symbol
        elif len(gene) > 2:
            gene_list.append(gene)
 
        # If they start different, it might have only a number (e.g., 'ABC1, 2, 3') so it needs to be appended
        elif gene.isdigit():
            gene_list.append(genes[0][:-1] + gene)
        
        # If the have only one letter (e.g., HTR1A,B,D,E,F,HTR5A)
        elif len(gene) == 1:
            gene_list.append(genes[0][:-1] + gene)
            
    return gene_list
    
def munge_reactome_gene(gene):
    """Process Reactome gene"""
    if "," in gene:
        return process_reactome_multiple_genes(gene.split(","))
        
    elif "/" in gene:
        return process_reactome_multiple_genes(gene.split("/"))
    
    return gene

def calculate_database_sets(nodes, database):
    """Calculate node sets for each modality in the database"""
    gene_nodes = set()
    mirna_nodes = set()
    metabolite_nodes = set()
    bp_nodes = set()
    
    for node in nodes:
        
        if isinstance(node, ListAbundance) or isinstance(node, Reaction) or not node.name:
            continue
                
        # Lower case name and strip quotes or white spaces
        name = node.name.lower().strip('"').strip()
        
        # Dealing with Genes/miRNAs
        if isinstance(node, CentralDogma):
            
            ##################
            # miRNA entities #
            ##################
            
            if name.startswith("mir"):
                
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            mirna_nodes.add(name.replace("mir-", "mir"))
                    else:
                        mirna_nodes.add(name.strip(' genes').replace("mir-", "mir"))
                        
                    continue
                
                mirna_nodes.add(name.replace("mir-", "mir"))
                
            ##################
            # Genes entities #
            ##################
            
            else:
                # Reactome preprocessing to flat multiple identifiers
                if database == 'reactome':
                    reactome_cell = munge_reactome_gene(name)
                    if isinstance(reactome_cell, list):
                        for name in reactome_cell:
                            if name in BLACK_LIST_REACTOME: # Filter entities in black list
                                continue
                            elif name.startswith("("): # remove redundant parentheses
                                name = name.strip("(").strip(")")
                                
                            gene_nodes.add(name)
                    else:
                        gene_nodes.add(name)
                    continue
                    
                # WikiPathways and KEGG do not require any processing of genes
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue
                gene_nodes.add(name)
         
        #######################
        # Metabolite entities #
        #######################
        
        elif isinstance(node, Abundance):
            
            if database == 'wikipathways':
                # Biological processes that are captured as abundance in BEL since they were characterized wrong in WikiPathways
                if name in WIKIPATHWAYS_BIOL_PROCESS:
                    bp_nodes.add(name)
                    continue

                elif node.namespace in {'WIKIDATA', 'WIKIPATHWAYS', 'REACTOME'} and name not in WIKIPATHWAYS_METAB:
                    bp_nodes.add(name)
                    continue
                    
                # Fix naming in duplicate entity
                if name in WIKIPATHWAYS_NAME_NORMALIZATION:
                    name = WIKIPATHWAYS_NAME_NORMALIZATION[name]
                    
            elif database == 'reactome':
                # Curated proteins that were coded as metabolites
                if name in REACTOME_PROT:
                    gene_nodes.add(name)
                    continue
                
                # Flat multiple identifiers (this is not trivial because most of ChEBI names contain commas, 
                # so a clever way to fix some of the entities is to check that all identifiers contain letters)
                elif "," in name and all(
                    string.isalpha() 
                    for string in name.split(",")
                ):
                    for string in name.split(","):
                        metabolite_nodes.add(name)
                    continue
                    
            metabolite_nodes.add(name)

        #################################
        # Biological Processes entities #
        #################################
        
        elif isinstance(node, BiologicalProcess):
            if name.startswith('title:'):
                name = name[6:] # KEGG normalize
            
            bp_nodes.add(name)
        
    return gene_nodes, mirna_nodes, metabolite_nodes, bp_nodes

In [8]:
kegg_genes, kegg_mirna, kegg_metabolites, kegg_bps = calculate_database_sets(
    kegg_nodes, 'kegg'
)

reactome_genes, reactome_mirna, reactome_metabolites, reactome_bps = calculate_database_sets(
    reactome_nodes, 'reactome'
)
wikipathways_genes, wikipathways_mirna, wikipathways_metabolites, wikipathways_bps = calculate_database_sets(
    wikipathways_nodes, 'wikipathways'
)

####  Entity count for each modality in each database

In [9]:
print(f'Total number of genes: KEGG ({len(kegg_genes)}), Reactome ({len(reactome_genes)}), WikiPathways ({len(wikipathways_genes)})')

print(f'Total number of metabolites: KEGG ({len(kegg_metabolites)}), Reactome ({len(reactome_metabolites)}), WikiPathways ({len(wikipathways_metabolites)})')

print(f'Total number of miRNAs: KEGG ({len(kegg_mirna)}), Reactome ({len(reactome_mirna)}), WikiPathways ({len(wikipathways_mirna)})')

print(f'Total number of Biological Processes: KEGG ({len(kegg_bps)}), Reactome ({len(reactome_bps)}), WikiPathways ({len(wikipathways_bps)})')

Total number of genes: KEGG (7283), Reactome (6328), WikiPathways (3025)
Total number of metabolites: KEGG (4041), Reactome (2570), WikiPathways (454)
Total number of miRNAs: KEGG (149), Reactome (11), WikiPathways (68)
Total number of Biological Processes: KEGG (418), Reactome (2101), WikiPathways (102)


## Dataset label mapping to PathMe

In [10]:
def check_substrings(dataset_nodes, db_nodes):
    intersection_close = set()
    for entity in dataset_nodes:
        if isinstance(entity, tuple):
            for subentity in entity:
                for entity_db in db_nodes:
                    if entity_db in subentity or subentity in entity_db:
                        intersection_close.add(entity_db)
        else:
            for entity_db in db_nodes:
                if entity_db in entity or entity in entity_db:
                        intersection_close.add(entity_db)
    return intersection_close

### Dataset labels

In [11]:
kegg_all_omics_labels = set.union(kegg_mirna, kegg_metabolites, kegg_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(reactome_mirna, reactome_metabolites, reactome_genes)
 
all_omics_labels_dataset1 = set()
for labels in dataset1_omics_labels.values():
    all_omics_labels_dataset1.update(set(labels))
                                         
all_omics_labels_dataset1

{'',
 'uhrf2',
 'dctn6',
 'arhgap21',
 'txndc17',
 'pum2',
 'ncapg2',
 'mobkl1b',
 'c14orf49',
 'sod1',
 'znf57',
 'hbg2',
 'fgfbp3',
 'znf562',
 'ier3',
 'cers4',
 'pprc1',
 'cdk7',
 'ifi6',
 'ppp1r13l',
 'prmt6',
 'alanine',
 'chac2',
 'znf195',
 'anks4b',
 'eef1d',
 'cttn',
 'azgp1',
 'rtcd1',
 'ppt1',
 'soat2',
 's100a9',
 'prr14',
 'hif1a',
 'plrg1',
 'trim25',
 'tmem14a',
 'mrpl21',
 'hebp2',
 'cdipt',
 'maml1',
 'tbc1d24',
 'tyms',
 'mdm4',
 'tgm2',
 'bckdhb',
 'mtfr1',
 'ube2w',
 'samhd1',
 'c2orf68',
 'itih5',
 'c1s',
 'dus1l',
 'znf275',
 'klhdc2',
 'bcl9l',
 'aldh1b1',
 'ripk2',
 'klhl21',
 'clpx',
 'fbxo46',
 'dffb',
 'ttc13',
 'wdr41',
 'bcor',
 'pah',
 'glud1',
 'ddx27',
 'tmem65',
 'c11orf67',
 'coro1b',
 'mospd2',
 'abcg8',
 'exo1',
 'taldo1',
 'as3mt',
 'rbck1',
 'hlf',
 'cyp20a1',
 'hsamir320d',
 'prdx4',
 'decr1',
 'bbs2',
 'arl8a',
 'pbk',
 'cmtm6',
 'rfxap',
 'rusc2',
 'trim5',
 'gulp1',
 'mdk',
 'fam73a',
 'ttc27',
 'slc35e3',
 'dtx4',
 'ptprh',
 'amica1',
 'c12or

### KEGG

In [12]:
len(kegg_genes.intersection(dataset1_omics_labels['genes']))

1999

In [13]:
len(kegg_metabolites.intersection(dataset1_omics_labels['metabolite']))

8

In [14]:
print(len(kegg_mirna.intersection(dataset1_omics_labels['micrornas'])))
print(len(check_substrings(dataset1_omics_labels['micrornas'], kegg_mirna)))

0
20


In [15]:
kegg_mapping = kegg_all_omics_labels.intersection(all_omics_labels_dataset1)
len(kegg_mapping)

2007

In [16]:
print(len(kegg_mapping)/total_entities)

0.396405293304365


### Reactome

In [17]:
len(reactome_genes.intersection(dataset1_omics_labels['genes']))

1387

In [18]:
len(reactome_metabolites.intersection(dataset1_omics_labels['metabolite']))

2

In [19]:
print(len(reactome_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], reactome_mirna))

0


4

In [20]:
len(reactome_all_omics_labels.intersection(all_omics_labels_dataset1))

1393

### WikiPathways

In [21]:
len(wikipathways_genes.intersection(dataset1_omics_labels['genes']))

934

In [22]:
len(wikipathways_metabolites.intersection(dataset1_omics_labels['metabolite']))

11

In [23]:
print(len(wikipathways_mirna.intersection(dataset1_omics_labels['micrornas'])))
len(check_substrings(dataset1_omics_labels['micrornas'], wikipathways_mirna))

0


11

In [24]:
len(wikipathways_all_omics_labels.intersection(all_omics_labels_dataset1))

945

### All db mapping

In [25]:
all_omics_labels_db = set.union(kegg_all_omics_labels, reactome_all_omics_labels)

In [26]:
total_entities_db = len(all_omics_labels_db.intersection(all_omics_labels_dataset1))
print(total_entities_db)

print(total_entities_db/total_entities)

2479
0.48963065376259135


## Score Diffusion with diffuPy: Dataset as input + PathMe as background graph

In [27]:
from diffuPy.diffuse import diffuse

from diffuPy.matrix import Matrix, LaplacianMatrix

### Input  Matrix: Dataset

In [28]:
kegg_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
wikipathways_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)
reactome_all_omics_labels = set.union(wikipathways_mirna, wikipathways_metabolites, wikipathways_genes)

all_omics_labels = set.union(kegg_all_omics_labels, wikipathways_all_omics_labels, reactome_all_omics_labels) 

input_mat = Matrix(rows_labels=all_omics_labels, cols_labels=['Dataset 1'], init=1)

In [29]:
print(input_mat)


matrix  
  [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]] 
 row labels: 
  ['mt-atp8', 'pde4d', 'sod1', '2703410', 'ier3', 'perp', 'nkx3-2', 'pprc1', 'alanine', 'cdk7', 'ifi6', 'selp', 'palmitoyl-coa', 'aqp7p3', 'traf1', 'cttn', 'csnk1a1l', 'cyp2a7', 'hif1a', 'trim25', 'birc6', 'itga10', 'vim', '5-methoxyindoleacetate', 'suclg1', 'maml1', 'cdipt', 'hotair', 'tyms', 'mdm4', 'dlst', 'fgf21', 'cfhr4', 'ensg00000204490', 'siah2', 'uba1', 'c1s', 'yp_043213', 'dffb', 'ripk2', 'ensg00000207949', '2-deoxyguanosine', 'pah', 'app', 'glud1', 'wnt6', 'gja1', 'mga', 'podxl', 'fosb', '5-methyl thf', 'serotonin', 'il10ra', 'exo1', 'taldo1', 'adra2c', 'prkn', 'neurog1', 'gnb1', 'ctbp2', 'selenof', 'cyp20a1', 'inhba', 'eif2b2', 'gsta2', 'decr1', 'ebf1', 'l-tryptophan', 'elavl1', 'pbk', 'mapkapk5', 'ca2+', "2'-deoxyadenosine", 'glu', 'pi3k', 'ndn', 'tbkbp1', 'ensg00000274784', 'dtx4', 'thbs2', 'ctnnal1', 'ring1', '6.3.5.4', 'hand2', 'bmp1', 'slc22a7', 'pfas', 'mir140', 'cdk2', 'ox-ldl', 'mir199a1', 'rfc1', 'kcnip

### Kernel Matrix: PathMe Universe Graph

#### Graph Universe

In [30]:
def get_db_graph_universe(folder):
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    database_networks = [
        pybel.from_pickle(os.path.join(folder, path))
        for path in os.listdir(folder)
        if path.endswith('.pickle')
    ]
    
    return pybel.struct.union(database_networks)

def get_pathme_graph_universe():
    """Merge all the python pickles in a given folder and returns the corresponding BELGraph."""  
    if 'pathme_universe.pickle' in os.listdir(PATHME_DIR):
        return pybel.from_pickle(os.path.join(PATHME_DIR, 'pathme_universe.pickle'))

    else:
        kegg_graph = get_db_graph_universe(KEGG_BEL)
        reactome_graph = get_pathme_graph_universe(REACTOME_BEL)
        wikipathways_graph = get_pathme_graph_universe(WIKIPATHWAYS_BEL)
        
        pathme_universe = pybel.struct.union([kegg_graph, reactome_graph, wikipathways_graph])
        pathme_universe = pybel.struct.union(database_networks)
        pybel.to_pickle(pathme_universe, PATHME_DIR)
        return pathme_universe



In [31]:
pathme_graph_universe = get_pathme_graph_universe()

In [32]:
pathme_graph_universe.graph['pathway_id'] = 'pathme_graph_universe'

In [33]:
pathme_graph_universe.summarize()

Fc epsilon RI signaling pathway v1.0.0
Number of Nodes: 49602
Number of Edges: 97866
Network Density: 3.98E-05
Number of Components: 6570
Number of Warnings: 0


In [35]:
background_mat = LaplacianMatrix(pathme_graph_universe.to_undirected())

Rows labels empty, also columns (neither cols labels given) will be empty since duplicate labels is true.


In [36]:
background_mat


matrix  
  [[22  0  0 ...  0  0  0]
 [ 0 19  0 ...  0  0  0]
 [ 0  0  4 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  2  0]
 [ 0  0  0 ...  0  0  1]] 
 row labels: 
  [] 
 column labels: 
  [] 
 


### Compute diffusion scores

In [ ]:
diffuse(input_mat, 'gm', K = background_mat)